In [1]:
import pandas as pd # type: ignore
import os

# Data Products

## Extracting Data from Source Folder

In [2]:
# Creating a list with file names in the data_products folder
destination_path = 'data/data_products'
all_files = [file for file in os.listdir(destination_path) if file.endswith('.csv')] # Creating a list of file names in the path directory

# Create a list of dataframes for each files in the data_products folder
dataframes = []
for i in all_files:
    file = os.path.join(destination_path, i)
    df = pd.read_csv(file)
    dataframes.append(df)

# Concate all dataframes in the 'dataframes' lists
df_products_original = pd.concat(dataframes, ignore_index=True)

In [3]:
# Creating a copy to preserve original dataframes
df_products = df_products_original.copy()

df_products.head()

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,Unnamed: 0
0,Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/31UISB90sY...,https://www.amazon.in/Lloyd-Inverter-Convertib...,4.2,"2,255","₹32,999","₹58,990",NaN
1,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.2,"2,948","₹46,490","₹75,990",NaN
2,LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Inverter-Convertible-...,4.2,"1,206","₹34,490","₹61,990",NaN
3,LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.0,69,"₹37,990","₹68,990",NaN
4,Carrier 1.5 Ton 3 Star Inverter Split AC (Copp...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/41lrtqXPiW...,https://www.amazon.in/Carrier-Inverter-Split-C...,4.1,630,"₹34,490","₹67,790",NaN


## Cleaning and Transforming Data

In [4]:
# showing info of the Ingested data
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1103170 entries, 0 to 1103169
Data columns (total 10 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   name            1103170 non-null  object 
 1   main_category   1103170 non-null  object 
 2   sub_category    1103170 non-null  object 
 3   image           1103170 non-null  object 
 4   link            1103170 non-null  object 
 5   ratings         751582 non-null   object 
 6   no_of_ratings   751582 non-null   object 
 7   discount_price  980844 non-null   object 
 8   actual_price    1067544 non-null  object 
 9   Unnamed: 0      551585 non-null   float64
dtypes: float64(1), object(9)
memory usage: 84.2+ MB


Penyelidikan terhadap data. Untuk sejauh ini kita bisa lihat bahwa terdapat beberapa null value dalam beberapa kolom itu. Kita bisa melihat bahwa untuk kolom ```ratings, no_of_ratings, discount_price, actual_price``` yang seharusnya memiliki tipe data interger atau float. Terakhir, terdapat kolom ```Unnamed: 0``` yang perlu kita selidiki.

In [5]:
# Untuk melihat bentuk data tipe masing-masing kolom dan jumlah non-null values

df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1103170 entries, 0 to 1103169
Data columns (total 10 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   name            1103170 non-null  object 
 1   main_category   1103170 non-null  object 
 2   sub_category    1103170 non-null  object 
 3   image           1103170 non-null  object 
 4   link            1103170 non-null  object 
 5   ratings         751582 non-null   object 
 6   no_of_ratings   751582 non-null   object 
 7   discount_price  980844 non-null   object 
 8   actual_price    1067544 non-null  object 
 9   Unnamed: 0      551585 non-null   float64
dtypes: float64(1), object(9)
memory usage: 84.2+ MB


Lalu kita akan lanjut untuk melihat bagaimana cara menghandle numeric data type. Setelah diselidiki ditemukan beberapa non-numeric values dimasing-masing kolom. Oleh karena itu, melainkan menggunakan ```astype()``` method, kita menggunakan ```apply()``` method untuk mengubah values dari string (atau tipe data yang lain) menjadi ```int/float```. Untuk non-numeric kita akan ubah valuenya menjadi ```NaN``` atau `None` tapi juga akan tetap melihat kasus unik masing-masing kolom.

In [6]:
# Membuat fungsi untuk mengubah menjadi interger atau float

def getFloat(number):
    try:
        number = float(number)
        return number
    except:
        return None   

In [7]:
# melihat unique rating
df_products['ratings'].unique()

array(['4.2', '4.0', '4.1', '4.3', '3.9', '3.8', '3.5', nan, '4.6', '3.3',
       '3.4', '3.7', '2.9', '5.0', '4.4', '3.6', '2.7', '4.5', '3.0',
       '3.1', '3.2', '4.8', '4.7', '2.5', '1.0', '2.6', '2.8', '2.3',
       '1.7', 'Get', '1.8', '2.4', '2.0', '1.5', '4.9', '1.9', '2.2',
       '1.2', '2.1', '1.4', '1.6', '1.3', 'FREE', '₹2.99', '1.1', '₹70',
       4.5, 3.3, 3.7, 4.2, 4.3, 4.1, 3.8, 4.4, 4.0, 3.5, 3.9, '₹99', 3.4,
       3.6, 3.2, 5.0, 2.6, 4.7, 3.0, 3.1, 4.6, 1.0, 4.9, 2.9, 2.7, 2.2,
       2.5, 4.8, 2.0, 1.7, 1.9, 2.8, 2.4, 1.6, 2.3, 1.8, 1.4, 1.3, 2.1,
       1.5, '₹100', '₹68.99', '₹65'], dtype=object)

Setelah dilihat ditemukan bahwa terdapat beberapa `None` values tetapi juga error values seperti string ataupun numerical dalam bentuk mata uang. Kita akan mengubahnya menjadi `NaN`

In [8]:
df_products['ratings'] = df_products['ratings'].apply(getFloat)

In [9]:
# Menghilangkan ',' agar dapat diubah menjadi interger
df_products['no_of_ratings'] = df_products['no_of_ratings'].str.replace(',', '')

# Untuk bantu check non number valuesnya seperti apa. karena kolom 'no_of_ratings' memiliki unique values sangat banyak
def getNonNumber(number):
    try:
        number = float(number)
        return 0
    except:
        return number

test_norating = df_products['no_of_ratings'].apply(getNonNumber)
test_norating.unique()

# Karena mengetahui bahwa angka non-numeric values merupaka string error yang tidak dimiliki kolom lain, kita dapat mengubahnya menjadi nilai 0

array([0, 'Only 2 left in stock.', 'Only 1 left in stock.',
       'FREE Delivery by Amazon', 'Usually dispatched in 3 to 4 weeks.',
       'Usually dispatched in 5 to 6 days.',
       'Usually dispatched in 4 to 5 days.',
       'Usually dispatched in 6 to 7 days.',
       'Usually dispatched in 7 to 8 days.',
       'Usually dispatched in 11 to 12 days.',
       'Usually dispatched in 4 to 5 weeks.', 'Only 4 left in stock.',
       'Only 3 left in stock.', 'Usually dispatched in 1 to 2 months.',
       'Only 5 left in stock.',
       'This item will be released on August 14 2023.',
       'Usually dispatched in 3 to 5 days.',
       'Usually dispatched in 2 to 3 days.',
       'Usually dispatched in 9 to 10 days.',
       'Usually dispatched in 2 to 3 weeks.',
       'Usually dispatched in 8 to 9 days.'], dtype=object)

Kita menemukan bahwa banyak values yang tidak sesuai dengan berbentuk `string`

In [10]:
df_products['no_of_ratings'] = df_products['no_of_ratings'].apply(getFloat).astype('Int64')

In [11]:
# Bersihkan string agar dapat diubah
df_products['discount_price'] = df_products['discount_price'].str.replace('₹', '').str.replace(',', '')

# Karena bekerja dengan currency kita akan menggunaka datatype float
df_products['discount_price'] = df_products['discount_price'].astype(float)

In [12]:
# Bersihkan string agar dapat diubah
df_products['actual_price'] = df_products['actual_price'].str.replace('₹', '').str.replace(',', '')

# Karena bekerja dengan currency kita akan menggunaka datatype float
df_products['actual_price'] = df_products['actual_price'].astype(float)

Kita akan membuat kolom baru `current_price`, menggunakannya untuk analisis harga campuran (karena beberapa tidak memiliki diskon).

In [13]:
# Membuat fungsi untuk retrurn `discount_price` jika ada diskon, dan return 'actual_price' jika tidak ada.
def getCopyPrice(disc, actual):
    if pd.isnull(disc) and pd.notnull(actual):
        return actual
    else:
        return disc
    
# Membuat kolom 'current_price'
df_products['current_price'] = df_products.apply(lambda x: getCopyPrice(x['discount_price'], x['actual_price']), axis=1)

Setelah dilihat, ternyata memang beberapa row memiliki harga 0, kita akan mengubahnya untuk menjadi NaN dan akan memisahkannya dari tabel utama menjadi tabel tersendirinya.

In [14]:
def getNone(x):
    if x == 0:
        return None
    else:
        return x
    
df_products['current_price'] = df_products['current_price'].apply(getNone)

In [15]:
# Creating null dataframe
df_products_null = df_products[df_products['current_price'].isna()]

df_products_null

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,Unnamed: 0,current_price
76,LG 1.5 Ton 3 Star Hot & Cold DUAL Inverter Spl...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51hbo8yQ1E...,https://www.amazon.in/LG-Inverter-Convertible-...,4.0,265,NaN,NaN,NaN,NaN
100,Hitachi 1.5 Ton 5 Star Inverter Split AC (Copp...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/41AY1pk5oR...,https://www.amazon.in/Hitachi-Inverter-Copper-...,3.8,748,NaN,NaN,NaN,NaN
114,Panasonic 1.5 Ton 4 Star Wi-Fi Twin-Cool Inver...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/41Edvsb7Gh...,https://www.amazon.in/Panasonic-Conditioner-An...,4.5,195,NaN,NaN,NaN,NaN
144,Daikin 1.5 Ton 4 Star Hot & Cold Heat Pump Inv...,appliances,Air Conditioners,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Daikin-Inverter-Conditio...,3.7,27,NaN,NaN,NaN,NaN
146,LG 1.5 Ton 5 Star AI DUAL Inverter Wi-Fi Split...,appliances,Air Conditioners,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/LG-Inverter-Convertible-...,4.7,34,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1102671,IronMind Captains of Crush Hand Gripper,sports & fitness,All Exercise & Fitness,https://m.media-amazon.com/images/I/71xasbyrrS...,https://www.amazon.in/IronMind-Captains-Crush-...,4.8,2099,NaN,NaN,NaN,NaN
1102724,BIONFIT ON05M Patented Air Bike Exercise cycle...,sports & fitness,All Exercise & Fitness,https://m.media-amazon.com/images/I/71HdQzFtVo...,https://www.amazon.in/BIONFIT-Patented-Resista...,NaN,<NA>,NaN,NaN,NaN,NaN
1102765,Lotto Black PolyesterBlend Men's Pants (890718...,sports & fitness,All Exercise & Fitness,https://m.media-amazon.com/images/I/51u3zEqdnl...,https://www.amazon.in/Lotto-Black-PolyesterBle...,3.9,4,NaN,NaN,NaN,NaN
1102770,Sport Cushioned Socks Non Slip Grip for Basket...,sports & fitness,All Exercise & Fitness,https://m.media-amazon.com/images/I/71P1ax+52A...,https://www.amazon.in/Cushioned-Basketball-Cyc...,4.0,1,NaN,NaN,NaN,NaN


In [16]:
# Melakukan drop terhadap null values dalam kolom 'current_price'
df_products.dropna(subset=['current_price'], inplace=True)

Karena kita sudah mentransformasi masing-masing kolom menjadi data type yang sesuai kita akan menyingkirkan kolom ```Unnamed: 0``` karena dinilai tidak relevan dengan kebutuh analisis data.

In [17]:
df_products = df_products.drop('Unnamed: 0', axis=1)

In [18]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1067538 entries, 0 to 1103169
Data columns (total 10 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   name            1067538 non-null  object 
 1   main_category   1067538 non-null  object 
 2   sub_category    1067538 non-null  object 
 3   image           1067538 non-null  object 
 4   link            1067538 non-null  object 
 5   ratings         725588 non-null   float64
 6   no_of_ratings   725532 non-null   Int64  
 7   discount_price  980844 non-null   float64
 8   actual_price    1067538 non-null  float64
 9   current_price   1067538 non-null  float64
dtypes: Int64(1), float64(4), object(5)
memory usage: 90.6+ MB


## Data Demography

Dari kolom yang kita miliki, terdapat 2 kolom kategori yang bisa digunakan untuk melihat pembagian kelompok dengan aggregate functions yaitu `main_category` dan `sub_category`. Lalu kita memiliki kolom values yang bisa mendapatkan insight dari nilai `ratings`, `discount_price` yang akan kita olah untuk melihat besar diskon, dan `actual_price` untuk mendapatkan insight mengenai harga.

In [19]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1067538 entries, 0 to 1103169
Data columns (total 10 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   name            1067538 non-null  object 
 1   main_category   1067538 non-null  object 
 2   sub_category    1067538 non-null  object 
 3   image           1067538 non-null  object 
 4   link            1067538 non-null  object 
 5   ratings         725588 non-null   float64
 6   no_of_ratings   725532 non-null   Int64  
 7   discount_price  980844 non-null   float64
 8   actual_price    1067538 non-null  float64
 9   current_price   1067538 non-null  float64
dtypes: Int64(1), float64(4), object(5)
memory usage: 90.6+ MB


In [20]:
# Melihat jumlah unique values dalam 'main_category'
print(df_products['main_category'].nunique())

# Melihat unique kolom 'main_category'
df_products['main_category'].unique()

20


array(['appliances', 'toys & baby products', "men's shoes",
       'bags & luggage', 'car & motorbike', 'stores', "men's clothing",
       'home & kitchen', "women's clothing", 'accessories',
       'tv, audio & cameras', 'beauty & health', 'sports & fitness',
       "kids' fashion", 'music', 'grocery & gourmet foods',
       'pet supplies', "women's shoes", 'home, kitchen, pets',
       'industrial supplies'], dtype=object)

In [21]:
# Melihat jumlah unique values dalam 'sub_category'
print(df_products['sub_category'].nunique())

# Melihat unique kolom 'sub_category'
df_products['sub_category'].unique()

112


array(['Air Conditioners', 'Nursing & Feeding', 'Casual Shoes', 'Wallets',
       'Car Accessories', 'Diapers', 'Amazon Fashion', 'T-shirts & Polos',
       'Bedroom Linen', 'Toys & Games', 'Clothing', 'Sunglasses',
       'Travel Accessories', 'Headphones', 'Household Supplies',
       'Innerwear', 'Lingerie & Nightwear', 'Cardio Equipment',
       'Toys Gifting Store', 'Personal Care Appliances',
       'Strength Training', 'Shirts', 'Home Décor', 'Sportswear',
       'Backpacks', 'Car & Bike Care', 'Travel Duffles',
       'Camping & Hiking', "Kids' Shoes",
       'Musical Instruments & Professional Audio', 'Diet & Nutrition',
       "Men's Fashion", 'Home Furnishing', 'Ethnic Wear',
       'Camera Accessories', 'Speakers', 'Home Improvement', 'Watches',
       'Cricket', "Kids' Watches", 'All Grocery & Gourmet Foods',
       'Bags & Luggage', 'Washing Machines', 'Sewing & Craft Supplies',
       'Refrigerators', 'Jewellery', 'All Pet Supplies',
       'Fashion Sandals', 'Rucksacks'

In [22]:
# Melihat rata-rata raing keseluruhan
df_products['ratings'].mean()

np.float64(3.831511546497462)

In [23]:
# Melihat harga termahal denga max() dan kolom 'current_price'
df_products['current_price'].max()

np.float64(1249990.0)

In [24]:
# Untuk melihat secara keseluruhan

df_products.describe()

,ratings,no_of_ratings,discount_price,actual_price,current_price
count,725588.000000,725532.0,9.808440e+05,1.067538e+06,1.067538e+06
mean,3.831512,835.049675,2.623161e+03,2.311141e+04,2.872740e+03
std,0.754401,8609.132096,9.458191e+03,1.355085e+07,9.565796e+03
min,1.000000,1.0,8.000000e+00,4.000000e+00,4.000000e+00
25%,3.500000,4.0,3.890000e+02,9.900000e+02,3.990000e+02
50%,3.900000,20.0,6.790000e+02,1.599000e+03,6.990000e+02
75%,4.300000,133.0,1.399000e+03,2.999000e+03,1.549000e+03
max,5.000000,589547.0,1.249990e+06,9.900000e+09,1.249990e+06


In [25]:
# Include kolom dengan object data type

df_products.describe(include=object)

,name,main_category,sub_category,image,link
count,1067538,1067538,1067538,1067538,1067538
unique,384825,20,112,449205,533769
top,Zeya Yellow Gold Ring,accessories,Men's Fashion,https://m.media-amazon.com/images/I/51uEPldT42...,https://www.amazon.in/Mothercare-Carrot-Regula...
freq,1370,222342,38086,6088,2


### Analisis ratings
Melakukan agregasi dengan nilai 'ratings' dengan melakukan groupby() dengan 'main_category' dan 'sub_category'.

In [26]:
# Mengagregat value ratings dengan pembagian kelompok kolom 'main_category'

df_products.groupby('main_category')['ratings'].agg(['mean', 'min', 'max', 'count']).sort_values(by='mean', ascending=False)

,mean,min,max,count
main_category,,,,
grocery & gourmet foods,4.071948,1.0,5.0,5996
pet supplies,4.029716,1.0,5.0,2820
toys & baby products,4.029163,1.0,5.0,10424
beauty & health,4.002551,1.0,5.0,15288
home & kitchen,3.977507,1.0,5.0,26408
music,3.960684,1.0,5.0,1638
stores,3.942651,1.0,5.0,55914
bags & luggage,3.935840,1.0,5.0,7182
accessories,3.903133,1.0,5.0,132220


In [27]:
# Mengagregat value ratings dengan pembagian kelompok kolom 'main_category' & sub category

df_products.groupby(['main_category', 'sub_category'])['ratings'].agg(['mean', 'min', 'max', 'count']).sort_values(by='mean', ascending=False)

,,mean,min,max,count
main_category,sub_category,,,,
toys & baby products,International Toy Store,4.572222,4.0,4.9,36
beauty & health,Value Bazaar,4.304545,3.7,4.6,132
grocery & gourmet foods,All Grocery & Gourmet Foods,4.176383,2.0,5.0,1880
beauty & health,Health & Personal Care,4.173901,2.4,5.0,2184
toys & baby products,Strollers & Prams,4.126721,1.0,5.0,494
...,...,...,...,...,...
car & motorbike,Car Electronics,3.507153,1.0,5.0,1454
"tv, audio & cameras",Headphones,3.498524,1.0,5.0,13278
beauty & health,Personal Care Appliances,3.444898,1.0,5.0,686


Kita dapat melihat bahwa 'grocery & gourmet foods' merupakan pemiliki rata-rata `main_category` paling tinggi dalam rating. Sedangkan pada `sub_category` dipuncaki oleh 'International Toy Store'

### Prices

Melakukan agregasi dengan nilai 'prices' dengan melakukan groupby() dengan 'main_category' dan 'sub_category'

In [28]:
# Mengagregat value current_price dengan pembagian kelompok kolom 'main_category'.

df_products.groupby('main_category')['current_price'].agg(['mean', 'min', 'max', 'count']).sort_values(by='mean', ascending=False)

,mean,min,max,count
main_category,,,,
accessories,5615.944755,4.00,499999.00,222342
appliances,5463.668770,20.00,230000.00,62652
music,4004.035453,50.00,76900.00,2076
stores,3923.041826,26.15,406009.00,64668
"home, kitchen, pets",3643.705882,1199.00,8541.00,34
"tv, audio & cameras",3427.548656,10.00,1249990.00,132710
men's shoes,2600.654225,25.00,66649.00,109726
bags & luggage,2317.375915,10.00,140000.00,19964
industrial supplies,2309.831312,35.94,144856.00,8020


In [29]:
# Mengagregat value 'current_prices' dengan menambahkan pembagian kolom 'sub_category'

df_products.groupby(['main_category', 'sub_category'])['current_price'].agg(['mean', 'min', 'max', 'count']).sort_values(by='mean', ascending=False)

,,mean,min,max,count
main_category,sub_category,,,,
"tv, audio & cameras",Televisions,39849.728827,100.0,1249990.0,1466
appliances,Air Conditioners,38725.549240,199.0,128800.0,1000
accessories,Gold & Diamond Jewellery,29929.124631,110.0,468012.0,31486
sports & fitness,Cardio Equipment,28846.721122,235.0,225250.0,392
appliances,Refrigerators,19792.077584,99.0,189200.0,3278
...,...,...,...,...,...
grocery & gourmet foods,"Coffee, Tea & Beverages",498.725428,8.0,7900.0,2568
beauty & health,Beauty & Grooming,481.695149,19.0,9200.0,3830
home & kitchen,Sewing & Craft Supplies,438.800707,35.0,15999.0,2518


Kita dapat melihat bahwa 'Accesories' merupakan pemiliki rata-rata `main_category` paling tinggi dalam `current_price`. Sedangkan pada `sub_category` dipuncaki oleh 'Televisions'

### Discount

Melakukan agregasi dengan nilai 'discount' dengan melakukan groupby() dengan 'main_category' dan 'sub_category'

In [30]:
# Membuat kolom discount

df_products['discount'] = df_products['actual_price'] - df_products['current_price']

In [31]:
df_products.groupby('main_category')['discount'].agg(['mean', 'max']).sort_values(by='mean', ascending=False)

,mean,max
main_category,,
home & kitchen,684923.029268,9.899999e+09
sports & fitness,6333.442432,6.108230e+07
appliances,2668.417563,3.700000e+05
accessories,2520.361441,7.939910e+05
"tv, audio & cameras",2298.170362,3.449100e+05
bags & luggage,1730.531382,9.600000e+04
music,1617.782216,5.244100e+04
men's shoes,1252.831183,2.695600e+04
industrial supplies,1136.832185,4.505000e+04


In [32]:
df_products.groupby(['main_category', 'sub_category'])['discount'].agg(['mean', 'sum', 'count']).sort_values(by='mean', ascending=False)

,,mean,sum,count
main_category,sub_category,,,
home & kitchen,Garden & Outdoors,9.017007e+06,1.980135e+10,2196
sports & fitness,Running,6.834569e+04,1.237057e+08,1810
"tv, audio & cameras",Televisions,2.238134e+04,3.281104e+07,1466
sports & fitness,Cardio Equipment,1.857530e+04,7.281519e+06,392
appliances,Air Conditioners,1.608968e+04,1.608968e+07,1000
...,...,...,...,...
beauty & health,Health & Personal Care,2.381061e+02,5.228810e+05,2196
grocery & gourmet foods,Snack Foods,1.970310e+02,4.114006e+05,2088
beauty & health,Value Bazaar,1.579241e+02,2.084598e+04,132


Kita dapat melihat bahwa 'Home & Kitchen' merupakan pemiliki rata-rata `main_category` paling tinggi dalam `discount`. hal ini konsisten dengan pada `sub_category` dipuncaki oleh 'Garden & Outdoors', yang memiliki `main_category` yang sama.

### Discount Percentage

Melakukan agregasi dengan nilai 'discount_percentage' dengan melakukan groupby() dengan 'main_category' dan 'sub_category'. Hal ini untuk mengukur besaran persentase diskon berada di kategori dimana.

In [33]:
# Membuat kolom 'discount percentage'

df_products['discount_percentage'] = (df_products['actual_price'] - df_products['current_price'])/df_products['actual_price']

In [34]:
df_products.groupby('main_category')['discount_percentage'].agg(['mean', 'max']).sort_values(by='mean', ascending=False)

,mean,max
main_category,,
women's clothing,0.565146,0.949703
accessories,0.505547,0.999000
"tv, audio & cameras",0.470715,0.991559
home & kitchen,0.458333,1.000000
bags & luggage,0.457284,0.999000
sports & fitness,0.445736,0.999989
car & motorbike,0.433778,0.979098
kids' fashion,0.422090,0.959111
men's clothing,0.412937,0.945541


In [35]:
df_products.groupby(['main_category', 'sub_category'])['discount_percentage'].agg(['mean', 'sum']).sort_values(by='mean', ascending=False)

mean           sum
main_category           sub_category                                       
accessories             Fashion & Silver Jewellery   0.643045  24289.093307
women's clothing        Ethnic Wear                  0.627406  23551.569500
                        Clothing                     0.595726  22676.908288
accessories             Jewellery                    0.575426  21667.077817
women's clothing        Western Wear                 0.558816  21251.765064
...                                                       ...           ...
grocery & gourmet foods All Grocery & Gourmet Foods  0.176166    336.124467
home, kitchen, pets     Refurbished & Open Box       0.172337      5.859442
stores                  Sportswear                   0.148848   2118.100738
grocery & gourmet foods Coffee, Tea & Beverages      0.147204    378.018619
toys & baby products    International Toy Store      0.121031      5.567447

[112 rows x 2 columns]

Kita dapat melihat bahwa 'Womens Cloting' merupakan pemiliki rata-rata `main_category` paling tinggi dalam `current_price`. Sedangkan pada `sub_category` dipuncaki oleh 'Fashion & Silver jewelry'